In [1]:
import folium
import pandas as pd
import numpy as np

import json
import requests # you need this module to make an API call

# Import the required library
from geopy.geocoders import Nominatim

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [5]:
df_jobs = pd.read_json('jobs.json')

In [11]:
df_jobs.Location.unique()

array(['Los Angeles', 'New York', 'San Francisco', 'Washington DC',
       'Boston', 'Seattle', 'Detroit', 'Austin', 'Houston',
       'Philadelphia', 'New Orleons', 'Baltimore', 'Dallas'], dtype=object)

In [26]:
df_jobs.nunique()

Id                         27005
Job Title                  21644
Job Experience Required      246
Key Skills                 25279
Role Category                204
Location                      13
Functional Area               69
Industry                     122
Role                         629
dtype: int64

In [27]:
df_jobs.shape

(27005, 9)

In [12]:
df_jobs.Location.value_counts()

Location
Washington DC    5316
Detroit          3945
Seattle          3375
Houston          3339
New York         3226
Boston           2966
Baltimore        1263
Dallas           1208
New Orleons       817
Los Angeles       640
San Francisco     435
Austin            434
Philadelphia       41
Name: count, dtype: int64

In [9]:


print("The latitude of the location is: ", location.latitude)
print("The longitude of the location is: ", location.longitude)

The latitude of the location is:  29.9759983
The longitude of the location is:  -90.0782127


In [13]:
df_jobs.loc[df_jobs['Location'] == 'New Orleons', 'Location'] = 'New Orleans'

In [14]:
df_jobs.Location.value_counts()

Location
Washington DC    5316
Detroit          3945
Seattle          3375
Houston          3339
New York         3226
Boston           2966
Baltimore        1263
Dallas           1208
New Orleans       817
Los Angeles       640
San Francisco     435
Austin            434
Philadelphia       41
Name: count, dtype: int64

In [19]:
geolocator = Nominatim(user_agent="MyApp")

locations = []
longs = []
lats = []

for i in df_jobs.Location.unique():
    locations.append(i)
    geolocation = geolocator.geocode(i)
    if geolocation != None:
        lats.append(geolocation.latitude)
        longs.append(geolocation.longitude)
    

In [20]:
len(locations)

13

In [24]:
lats

[34.0536909,
 40.7127281,
 37.7790262,
 38.8950368,
 42.3554334,
 47.6038321,
 42.3315509,
 30.2711286,
 29.7589382,
 39.9527237,
 29.9759983,
 39.2908816,
 32.7762719]

In [40]:
df_njobs = (df_jobs
 .groupby('Location')
 .count()['Id']
 .reset_index()
 .rename(columns={'Id':'n_of_jobs'})
)

In [32]:
df_jobs[df_jobs.Location=='Austin'].shape

(434, 9)

In [33]:
df_jobs_maps = pd.DataFrame(
    {
        'Location': locations,
        'Longitude': longs,
        'Latitude': lats
    }
)

df_jobs_maps

,Location,Longitude,Latitude
0,Los Angeles,-118.242766,34.053691
1,New York,-74.006015,40.712728
2,San Francisco,-122.419906,37.779026
3,Washington DC,-77.036543,38.895037
4,Boston,-71.060511,42.355433
5,Seattle,-122.330062,47.603832
6,Detroit,-83.046640,42.331551
7,Austin,-97.743700,30.271129
8,Houston,-95.367697,29.758938
9,Philadelphia,-75.163526,39.952724


In [48]:
df_jobs_maps = df_jobs_maps.merge(df_njobs, on = 'Location')

In [51]:
df_jobs_maps.sort_values('n_of_jobs', ascending = False)

,Location,Longitude,Latitude,n_of_jobs
3,Washington DC,-77.036543,38.895037,5316
6,Detroit,-83.046640,42.331551,3945
5,Seattle,-122.330062,47.603832,3375
8,Houston,-95.367697,29.758938,3339
1,New York,-74.006015,40.712728,3226
4,Boston,-71.060511,42.355433,2966
11,Baltimore,-76.610759,39.290882,1263
12,Dallas,-96.796856,32.776272,1208
10,New Orleans,-90.078213,29.975998,817
0,Los Angeles,-118.242766,34.053691,640


In [139]:
m = folium.Map(location=[42, -102], zoom_start=4)

In [153]:
import math

for city in df_jobs_maps.itertuples():
    local_deformation = math.cos(city.Latitude * math.pi / 180)
    folium.Circle(
        location=[city.Latitude, city.Longitude],
        popup='%s %.0f jobs' % (city.Location, city.n_of_jobs),
        tooltip= '%s %.0f jobs' % (city.Location, city.n_of_jobs),
        radius=city.n_of_jobs * 80.0 * local_deformation,
        color='crimson',
        fill=True,
        fill_color='crimson'
    ).add_to(m)
    folium.Marker(
        location=[city.Latitude, city.Longitude],
        popup='%s %.0f jobs' % (city.Location, city.n_of_jobs)
    ).add_to(m)

In [154]:
m